# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,19.98,69,53,3.57,PN,1690237058
1,1,lachi,33.3833,71.3386,31.75,61,100,0.34,PK,1690237059
2,2,port-aux-francais,-49.3500,70.2167,3.96,72,100,11.16,TF,1690237059
3,3,vernon,50.2581,-119.2691,27.56,36,81,2.06,CA,1690237059
4,4,yellowknife,62.4560,-114.3525,26.29,34,100,4.63,CA,1690236901


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
mplot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    size = 'Humidity',
    frame_width = 700,
    frame_height = 600,
    alpha = 0.5,
    xlabel = "Longitude (Degrees)",
    ylabel = "Latitude (Degrees)"
)

# Display the map
mplot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
limit = 10
ideal_df = city_data_df[(city_data_df['Max Temp'] >= 20) 
                        & (city_data_df['Max Temp'] <= 27) 
                        & (city_data_df['Wind Speed'] < 4.5)
                        & (city_data_df['Cloudiness'] == 0)].head(limit)

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,mishan,45.5500,131.8833,21.73,85,0,2.83,CN,1690237063
75,75,hami,42.8000,93.4500,26.50,25,0,4.05,CN,1690237069
126,126,shingu,33.7333,135.9833,26.05,75,0,0.83,JP,1690237076
139,139,dowlatabad,32.7998,51.6955,26.82,14,0,3.09,IR,1690237079
211,211,sivrice,38.4482,39.3084,22.56,36,0,3.60,TR,1690237096
272,272,dryden,49.7833,-92.7503,26.16,36,0,3.09,CA,1690237109
277,277,taghazout,30.5426,-9.7112,21.43,82,0,3.10,MA,1690237112
281,281,shizuishan,39.2333,106.7694,25.41,44,0,4.01,CN,1690237115
336,336,urumqi,43.8010,87.6005,24.53,34,0,1.00,CN,1690236969
390,390,kilju,40.9642,129.3278,24.20,78,0,1.78,KP,1690237132


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
29,mishan,CN,45.5500,131.8833,85,
75,hami,CN,42.8000,93.4500,25,
126,shingu,JP,33.7333,135.9833,75,
139,dowlatabad,IR,32.7998,51.6955,14,
211,sivrice,TR,38.4482,39.3084,36,
272,dryden,CA,49.7833,-92.7503,36,
277,taghazout,MA,30.5426,-9.7112,82,
281,shizuishan,CN,39.2333,106.7694,44,
336,urumqi,CN,43.8010,87.6005,34,
390,kilju,KP,40.9642,129.3278,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mishan - nearest hotel: XiLong
hami - nearest hotel: Хами
shingu - nearest hotel: ホテルニューパレス
dowlatabad - nearest hotel: هتل آفتاب اصفهان
sivrice - nearest hotel: No hotel found
dryden - nearest hotel: Holiday Inn Express
taghazout - nearest hotel: Riad Taghazout
shizuishan - nearest hotel: No hotel found
urumqi - nearest hotel: 百时快捷酒店红山店
kilju - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
29,mishan,CN,45.5500,131.8833,85,XiLong
75,hami,CN,42.8000,93.4500,25,Хами
126,shingu,JP,33.7333,135.9833,75,ホテルニューパレス
139,dowlatabad,IR,32.7998,51.6955,14,هتل آفتاب اصفهان
211,sivrice,TR,38.4482,39.3084,36,No hotel found
272,dryden,CA,49.7833,-92.7503,36,Holiday Inn Express
277,taghazout,MA,30.5426,-9.7112,82,Riad Taghazout
281,shizuishan,CN,39.2333,106.7694,44,No hotel found
336,urumqi,CN,43.8010,87.6005,34,百时快捷酒店红山店
390,kilju,KP,40.9642,129.3278,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotels_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    size = 'Humidity',
    frame_width = 700,
    frame_height = 600,
    alpha = 0.5,
    xlabel = "Longitude (Degrees)",
    ylabel = "Latitude (Degrees)",
    hover_cols = ['Hotel Name','Country']
)

# Display the map
hotels_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)